In [1]:
import numpy as np
import pydot
import graphviz
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.layers import Embedding, LSTM, Add
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import sentence_transformers 
import joblib
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython import display
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tqdm.notebook import tqdm
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import os

In [2]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


# Create the dataframe for train and test

In [5]:
# function to load documents into memory
def load_doc(filename):
    # Open file to read
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [6]:
# create dataframe from cleaned descriptions.txt file. pass file path to function to create
def get_dataframe(folder_path):
    dataframe = load_doc(folder_path)
    dataframe = dataframe.split('\n')

    l_all =[]
    for n in dataframe:
        line = {"id": n.split(" ")[0],
            'value': "startsequence " + " ".join(n.split(" ")[1:]) + " endsequence"}
        l_all.append(line)

    df_all = pd.DataFrame(l_all)

    t = Tokenizer()
    t.fit_on_texts(df_all.value)

    df_all["value_tokenized"] = t.texts_to_sequences(df_all.value)

    train_ids = np.unique(df_all.id)[:int(0.8*len(np.unique(df_all.id)))]
    test_ids = np.unique(df_all.id)[int(0.8*len(np.unique(df_all.id))):]

    df_train = df_all[df_all.id.isin(train_ids)]
    df_test = df_all[df_all.id.isin(test_ids)]

    return df_train,df_test,t

In [7]:
X_train,X_test,t = get_dataframe("../raw_data/descriptions.txt")

# CNN model

In [8]:
CNN_model = EfficientNetB0(
    include_top=False, # Whether to include the fully-connected layer at the top of the network
    weights='imagenet', # pre-trained weights on ImageNet
    input_tensor=None,
    input_shape= (256,256,3), # It should have exactly 3 inputs channels
    pooling=None # Optional pooling mode for feature extraction when include_top is False
)

2022-06-16 13:31:25.811100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 13:31:25.819980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 13:31:25.820339: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 13:31:25.821463: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Load the model

In [10]:
model = joblib.load('../output/model04_jlib')

2022-06-14 18:12:04.336867: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:12:04.350811: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:12:04.865927: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:12:04.879401: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:12:05.270698: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_sh

In [72]:
model01 = joblib.load('../output/model01_jlib')

2022-06-14 18:57:26.436858: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:57:26.449077: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:57:26.910428: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:57:26.922686: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 18:57:26.978988: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_sh

In [85]:
model02 = joblib.load('../output/model02_jlib')

2022-06-14 19:07:32.228374: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 19:07:32.855926: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 19:07:32.868278: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 19:07:33.550699: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-06-14 19:07:33.563611: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 5 outputs but the _output_sh

# Predict the results

In [9]:
def CNN_output(file_id):
    
    CNN_model = EfficientNetB0(
    include_top=False, # Whether to include the fully-connected layer at the top of the network
    weights='imagenet', # pre-trained weights on ImageNet
    input_tensor=None,
    input_shape= (256,256,3), # It should have exactly 3 inputs channels
    pooling=None # Optional pooling mode for feature extraction when include_top is False
    )
    test_img_path = f"../raw_data/pics/Flicker8k_Dataset/{file_id}"
    
    test_img= image.load_img(test_img_path, target_size=(256,256,3))
    x_test = image.img_to_array(test_img)
    x_test = np.expand_dims(x_test, axis=0)
    test = CNN_model.predict(x_test)
    return test

In [10]:
def predic_seq(img_encoded,model):   
    inputs_seq='startsequence'
    inputs_seq = [2]
    for i in range(36):
        inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=36)
        y_pre = model.predict([img_encoded,inputs_seq_model])
        next_word = y_pre.argmax()
        if next_word == 3:
            break
        inputs_seq.append(next_word)
    return inputs_seq

In [11]:
def output_sentence(inputs_seq):
    sentence  = []
    b = t.word_index
    for number in inputs_seq:     
        sentence.append(list(b.keys())[list(b.values()).index(number)])
        final_sentence  = ' '.join(word for word in sentence[1:])
    return final_sentence
    # return test_img

# Get the score

In [2]:
model_doc2vec= SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')


In [3]:
sentence = ['i am very good']

In [4]:
aaa= model_doc2vec.encode(sentence)

In [12]:
def rand_sequ(X_test,numbers,model):
    X_test.value = X_test.value.str.replace('startsequence ','')
    X_test.value = X_test.value.str.replace(' endsequence','')
    X_test =  X_test.drop(['value_tokenized'], axis=1)
    id_list = X_test.id.tolist()
    cap_list =X_test.value.tolist()
    img_list = id_list 
    capt_list = cap_list
    total_sentence = []
    image_id =[]
    rand_img_index = np.random.randint(1619, size=numbers)
    for i in rand_img_index:
        cnn_out = CNN_output(img_list[i]+'.jpg')
        seq_out = predic_seq(cnn_out,model)
        sent_out = output_sentence(seq_out)
        total_sentence.append(sent_out)
        image_id.append(img_list[i])
    captions = []
    for imgs in image_id:
        for index, row in X_test.iterrows():
            if row['id'] in imgs:
                captions.append(row['value'])
    predict_m = np.repeat(total_sentence,5)
    return captions, predict_m

In [86]:
captions, predict_m =rand_sequ(X_test,100,model02)

In [88]:
len(captions)

500

In [89]:
c= model_doc2vec.encode(captions)

In [90]:
d =model_doc2vec.encode(predict_m)

In [91]:
c.shape

(500, 512)

In [13]:
def score_ave(c,d,shape):
   
    score_total =[]

    for i in range(len(c)):
        score = sentence_transformers.util.cos_sim(c[i],d[i])
        score_total.append(score)
    score_total_np = np.array(score_total)
    score_total_final = score_total_np.reshape(5,shape)
        
    score_avr = []
    for i in range(len(score_total_final)):
        score_avr.append(score_total_final[i].mean())
    score_overall = np.array(score_avr)
    return score_overall.mean()

In [93]:
score_ave(c,d,100)


0.41559735

In [83]:
model_6k = 0.4247114

In [94]:
model_6k01 = 0.41559735

In [96]:
baseline_model = 0.13634846

# Try 30k model

In [13]:
import numpy as np
import pickle
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Embedding, Add, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
def model_builder():
    cap_len = 25
    vocab_size = 19758
    embed_dim = 1024
    inputs2  = Input(shape=(cap_len,),name="captions")
    embed_layer = Embedding(vocab_size+3, embed_dim, mask_zero=True)(inputs2)
    input_encoded = Input(shape=(8,8,1280),name="images_encoded")
    pooling = GlobalAveragePooling2D()(input_encoded)
    cnn_dense = Dense(embed_dim, activation='relu')(pooling)
    combine = Add()([embed_layer,cnn_dense])
    lstm_layer = LSTM(embed_dim)(combine)
    decoder = Dense(2048, activation='relu')(lstm_layer)
    outputs = Dense(vocab_size+1, activation='softmax')(decoder)
    lstm_model = Model(inputs=[input_encoded, inputs2], outputs=outputs)
    lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    return lstm_model

In [15]:
def load_model_cache():

    model = model_builder() #build empty model with the right architecure

    #Get Current directory Path File
    model.load_weights("model_run_30k_weights.h5") #load weights only from h5 file

    return model

In [16]:
model_30k = load_model_cache()

2022-06-16 13:32:25.582098: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 77.19MiB (rounded to 80941056)requested by op RandomUniform
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-16 13:32:25.582159: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-06-16 13:32:25.582173: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 79, Chunks in use: 79. 19.8KiB allocated for chunks. 19.8KiB in use in bin. 5.0KiB client-requested in use in bin.
2022-06-16 13:32:25.582181: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 65, Chunks in use: 65. 41.2KiB allocated for chunks. 41.2KiB in use in bin. 33.4KiB client-requested in use in bin.
2022-06-16 13:32:25.58218

ResourceExhaustedError: OOM when allocating tensor with shape[19761,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [87]:
def predict_caption(model,img_encoded):
    cap_len = 25
    index_file ='word_index.pkl'
    with open(index_file,'rb') as handle:
        word_index = pickle.load(handle)
    inputs_seq = [2]
    endsequence_token = word_index['endsequence']
    for i in range(cap_len):
        inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=25)
        y_pre = model.predict([img_encoded,inputs_seq_model])
        next_word = y_pre.argmax()
        if next_word == endsequence_token:
            break
        inputs_seq.append(next_word)
    sentence = []
    for number in inputs_seq:
        sentence.append(list(word_index.keys())[list(word_index.values()).index(number)])
    sentence = sentence [1:]
    prediction = ' '.join(word for word in sentence)
    return prediction

In [88]:
def rand_sequ_model30k(X_test,numbers,model):
    X_test.value = X_test.value.str.replace('startsequence ','')
    X_test.value = X_test.value.str.replace(' endsequence','')
    X_test =  X_test.drop(['value_tokenized'], axis=1)
    id_list = X_test.id.tolist()
    cap_list =X_test.value.tolist()
    img_list = id_list 
    capt_list = cap_list
    total_sentence = []
    image_id =[]
    rand_img_index = np.random.randint(1619, size=numbers)
    for i in rand_img_index:
        cnn_out = CNN_output(img_list[i]+'.jpg')
        sent_out = predict_caption(model,cnn_out)
        total_sentence.append(sent_out)
        image_id.append(img_list[i])
    captions = []
    for imgs in image_id:
        for index, row in X_test.iterrows():
            if row['id'] in imgs:
                captions.append(row['value'])
    predict_m = np.repeat(total_sentence,5)
    return captions, predict_m

In [89]:
captions_30k, predict_m_30k =rand_sequ_model30k(X_test,500,model_30k)

In [90]:
import pickle
pickle.dump(captions_30k,open('caption_30k.pkl','wb'))

In [91]:
pickle.dump(predict_m_30k,open('cpredict_m_30k.pkl','wb'))

In [20]:
len(captions_30k)

500

In [1]:
captions_30k = pickle.load('caption_30k.pkl')

NameError: name 'pickle' is not defined

In [21]:
c_30k= model_doc2vec.encode(captions_30k)

In [22]:
d_30k =model_doc2vec.encode(predict_m_30k)

In [19]:
def score_ave(c,d,shape):
   
    score_total =[]

    for i in range(len(c)):
        score = sentence_transformers.util.cos_sim(c[i],d[i])
        score_total.append(score)
    score_total_np = np.array(score_total)
    score_total_final = score_total_np.reshape(5,shape)
        
    score_avr = []
    for i in range(len(score_total_final)):
        score_avr.append(score_total_final[i].mean())
    score_overall = np.array(score_avr)
    return score_overall.mean()

In [24]:
score_ave(c_30k,d_30k,100)


0.68506813

In [25]:
model_30k_score = 0.68506813

# Comparison with OFA and 110kmodel

In [20]:
true_captions = ['three men sit in front of laptops on a wooden table in a classroom']

In [21]:
OFA_captions = ['three men sittig at a table working on laptops']

In [22]:
ourmodel_captions = ['a group of people sitting around a wooden table'] 

In [23]:
true_captions_vec = model_doc2vec.encode(true_captions)

In [24]:
OFA_captions_vec =  model_doc2vec.encode(OFA_captions)

In [25]:
ourmodel_captions_vec = model_doc2vec.encode(ourmodel_captions)

In [26]:
score1 = sentence_transformers.util.cos_sim(OFA_captions_vec,true_captions_vec)

In [27]:
score1

tensor([[0.7566]])

In [28]:
score2 = sentence_transformers.util.cos_sim(ourmodel_captions_vec,true_captions_vec)

In [29]:
score2

tensor([[0.5955]])

# Try 110k model

In [14]:
import numpy as np
import pickle
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Embedding, Add, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
def model_builder():
    cap_len = 25
    vocab_size = 28533
    embed_dim = 1024
    inputs2  = Input(shape=(cap_len,),name="captions")
    embed_layer = Embedding(vocab_size+3, embed_dim, mask_zero=True)(inputs2)
    input_encoded = Input(shape=(8,8,1280),name="images_encoded")
    pooling = GlobalAveragePooling2D()(input_encoded)
    cnn_dense = Dense(embed_dim, activation='relu')(pooling)
    combine = Add()([embed_layer,cnn_dense])
    lstm_layer = LSTM(embed_dim)(combine)
    decoder = Dense(2048, activation='relu')(lstm_layer)
    outputs = Dense(vocab_size+1, activation='softmax')(decoder)
    lstm_model = Model(inputs=[input_encoded, inputs2], outputs=outputs)
    lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
    return lstm_model

In [18]:
def load_model_cache():

    model = model_builder() #build empty model with the right architecure

    #Get Current directory Path File
    model.load_weights("20220614_118k_modelweights.h5") #load weights only from h5 file

    return model

In [19]:
model_110k = load_model_cache()

In [20]:
def predict_caption_big(model,img_encoded):
    cap_len = 25
    index_file ='word_index_118.pkl'
    with open(index_file,'rb') as handle:
        word_index = pickle.load(handle)
    inputs_seq = [2]
    endsequence_token = word_index['endsequence']
    for i in range(cap_len):
        inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=25)
        y_pre = model.predict([img_encoded,inputs_seq_model])
        next_word = y_pre.argmax()
        if next_word == endsequence_token:
            break
        inputs_seq.append(next_word)
    sentence = []
    for number in inputs_seq:
        sentence.append(list(word_index.keys())[list(word_index.values()).index(number)])
    sentence = sentence [1:]
    prediction = ' '.join(word for word in sentence)
    return prediction

In [23]:
def rand_sequ_model30k(X_test,numbers,model):
    X_test.value = X_test.value.str.replace('startsequence ','')
    X_test.value = X_test.value.str.replace(' endsequence','')
    X_test =  X_test.drop(['value_tokenized'], axis=1)
    id_list = X_test.id.tolist()
    cap_list =X_test.value.tolist()
    img_list = id_list 
    capt_list = cap_list
    total_sentence = []
    image_id =[]
    rand_img_index = np.random.randint(1619, size=numbers)
    for i in rand_img_index:
        cnn_out = CNN_output(img_list[i]+'.jpg')
        sent_out = predict_caption_big(model,cnn_out)
        total_sentence.append(sent_out)
        image_id.append(img_list[i])
    captions = []
    for imgs in image_id:
        for index, row in X_test.iterrows():
            if row['id'] in imgs:
                captions.append(row['value'])
    predict_m = np.repeat(total_sentence,5)
    return captions, predict_m

In [24]:
captions_110k, predict_m_110k =rand_sequ_model30k(X_test,100,model_110k)

In [25]:
len(captions_110k)

500

In [26]:
c_110k= model_doc2vec.encode(captions_110k)

In [27]:
d_110k =model_doc2vec.encode(predict_m_110k)

In [28]:
def score_ave(c,d,shape):
   
    score_total =[]

    for i in range(len(c)):
        score = sentence_transformers.util.cos_sim(c[i],d[i])
        score_total.append(score)
    score_total_np = np.array(score_total)
    score_total_final = score_total_np.reshape(5,shape)
        
    score_avr = []
    for i in range(len(score_total_final)):
        score_avr.append(score_total_final[i].mean())
    score_overall = np.array(score_avr)
    return score_overall.mean()

In [29]:
score_ave(c_110k,d_110k,100)

0.45181447

# Try Attention model

In [58]:
cap_len = 15
embed_dim = 1024
vocab_size = 10000

In [59]:
def model_builder_attention():
    inputs2  = Input(shape=(cap_len,),name="captions")
    embed_layer = Embedding(vocab_size+1, embed_dim, mask_zero=True)(inputs2)

    input_encoded = Input(shape=(8,8,1280),name="images_encoded")
    features = tf.keras.layers.Reshape((64, input_encoded.shape[-1]))(input_encoded)
    cnn_dense = Dense(embed_dim, activation='relu')(features)

    cnn_socre = tf.keras.layers.Dense(256)(cnn_dense)

    seq_context = LSTM(256)(embed_layer)
    seq_context = tf.expand_dims(seq_context, 1)
    seq_score = tf.keras.layers.Dense(256)(seq_context)

    combine_score = Add()([cnn_socre,seq_score])
    combine_score_activation = tf.nn.tanh(combine_score)

    attention_scores = Dense(1)(combine_score_activation)
    attention_weights = tf.nn.softmax(attention_scores, axis=1)

    heatmaped_image = tf.keras.layers.Multiply()([cnn_dense,attention_weights])
    pooling_heatmaped = tf.keras.layers.GlobalAveragePooling1D()(heatmaped_image)


    combine = Add()([embed_layer,pooling_heatmaped])
    lstm_layer = LSTM(embed_dim)(combine)
    decoder = Dense(2048, activation='relu')(lstm_layer)
    outputs = Dense(vocab_size, activation='softmax')(decoder)
    
    return Model(inputs=[input_encoded, inputs2], outputs=outputs)

In [62]:
lstm_model = model_builder_attention()
lstm_model.load_weights('20220614_attention_caplen25_modelweights.h5')

In [64]:
type(lstm_model)

keras.engine.functional.Functional

In [80]:
def predict_caption_attention(model,img_encoded):
    cap_len = 15
    index_file ='word_index_attention_10k.pkl'
    with open(index_file,'rb') as handle:
        word_index = pickle.load(handle)
    inputs_seq = [2]
    endsequence_token = word_index['endsequence']
    for i in range(cap_len):
        inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=15)
        y_pre = model.predict([img_encoded,inputs_seq_model])
        next_word = y_pre.argmax()
        if next_word == endsequence_token:
            break
        inputs_seq.append(next_word)
    sentence = []
    for number in inputs_seq:
        sentence.append(list(word_index.keys())[list(word_index.values()).index(number)])
    sentence = sentence [1:]
    prediction = ' '.join(word for word in sentence)
    return prediction

In [81]:
def image_to_array(path):
    loaded_img = image.load_img(path, target_size=(256,256,3))
    img_array = image.img_to_array(loaded_img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array
def cnn_builder():
    cnn_model = EfficientNetB0(include_top=False, weights='imagenet',
                           input_shape= (256,256,3))
    return cnn_model
def image_to_features(img_array):

    features = cnn_builder().predict(img_array)
    return features

In [82]:
img_array = image_to_array('IMG_9640.jpg')
img_encoded = image_to_features(img_array)
caption_attention = predict_caption_attention(lstm_model, img_encoded)
print(caption_attention)

men are playing music on a stage together while a couple of men converse behind


In [66]:
def rand_sequ_model_attention(X_test,numbers,model):
    X_test.value = X_test.value.str.replace('startsequence ','')
    X_test.value = X_test.value.str.replace(' endsequence','')
    X_test =  X_test.drop(['value_tokenized'], axis=1)
    id_list = X_test.id.tolist()
    cap_list =X_test.value.tolist()
    img_list = id_list 
    capt_list = cap_list
    total_sentence = []
    image_id =[]
    rand_img_index = np.random.randint(1619, size=numbers)
    for i in rand_img_index:
        cnn_out = CNN_output(img_list[i]+'.jpg')
        sent_out = predict_caption_attention(model,cnn_out)
        total_sentence.append(sent_out)
        image_id.append(img_list[i])
    captions = []
    for imgs in image_id:
        for index, row in X_test.iterrows():
            if row['id'] in imgs:
                captions.append(row['value'])
    predict_m = np.repeat(total_sentence,5)
    return captions, predict_m

In [68]:
captions_attention, predict_m_attention =rand_sequ_model_attention(X_test,100,lstm_model)

In [69]:
len(captions_attention)

500

In [70]:
c_attention= model_doc2vec.encode(captions_attention)

In [71]:
d_attention = model_doc2vec.encode(predict_m_attention)

In [72]:
def score_ave(c,d,shape):
   
    score_total =[]

    for i in range(len(c)):
        score = sentence_transformers.util.cos_sim(c[i],d[i])
        score_total.append(score)
    score_total_np = np.array(score_total)
    score_total_final = score_total_np.reshape(5,shape)
        
    score_avr = []
    for i in range(len(score_total_final)):
        score_avr.append(score_total_final[i].mean())
    score_overall = np.array(score_avr)
    return score_overall.mean()

In [73]:
score_ave(c_attention,d_attention,100)

0.678436